In [ ]:
!pip install pytorch-adapt

### Helper function and data for demo

In [1]:
import torch

from pytorch_adapt.utils.common_functions import get_lr


def print_optimizers_slim(adapter):
    for k, v in adapter.optimizers.items():
        print(f"{k}: {v.__class__.__name__} with lr={get_lr(v)}")


data = {
    "src_imgs": torch.randn(32, 1000),
    "target_imgs": torch.randn(32, 1000),
    "src_labels": torch.randint(0, 10, size=(32,)),
    "src_domain": torch.zeros(32),
    "target_domain": torch.zeros(32),
}

device = torch.device("cuda")

### Adapters Initialization

Models are usually the only required argument when initializing adapters. Optimizers are created using the default that is defined in the adapter. 

In [2]:
from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models

G = torch.nn.Linear(1000, 100)
C = torch.nn.Linear(100, 10)
D = torch.nn.Sequential(torch.nn.Linear(100, 1), torch.nn.Flatten(start_dim=0))
models = Models({"G": G, "C": C, "D": D})

adapter = DANN(models=models)
print_optimizers_slim(adapter)

G: Adam with lr=0.0001
C: Adam with lr=0.0001
D: Adam with lr=0.0001


### Modifying optimizers using the Optimizers container

We can use the Optimizers container if we don't want to use the defaults.

For example: SGD with lr 0.1 for all 3 models

In [3]:
from pytorch_adapt.containers import Optimizers

optimizers = Optimizers((torch.optim.SGD, {"lr": 0.1}))
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

G: SGD with lr=0.1
C: SGD with lr=0.1
D: SGD with lr=0.1


SGD with lr 0.1 for the G and C models only. The default optimizer will be used for D.

In [4]:
optimizers = Optimizers((torch.optim.SGD, {"lr": 0.1}), keys=["G", "C"])
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

G: SGD with lr=0.1
C: SGD with lr=0.1
D: Adam with lr=0.0001


SGD with lr 0.1 for G, and SGD with lr 0.5 for C

In [5]:
optimizers = Optimizers(
    {"G": (torch.optim.SGD, {"lr": 0.1}), "C": (torch.optim.SGD, {"lr": 0.5})}
)
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

G: SGD with lr=0.1
C: SGD with lr=0.5
D: Adam with lr=0.0001


You can also create the optimizers yourself and pass them into the Optimizers container

In [6]:
optimizers = Optimizers({"G": torch.optim.SGD(G.parameters(), lr=0.123)})
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

G: SGD with lr=0.123
C: Adam with lr=0.0001
D: Adam with lr=0.0001


### Adding LR Schedulers

LR schedulers can be added with the LRSchedulers container.

In [26]:
from pytorch_adapt.containers import LRSchedulers

optimizers = Optimizers((torch.optim.Adam, {"lr": 1}))
lr_schedulers = LRSchedulers(
    {
        "G": (torch.optim.lr_scheduler.ExponentialLR, {"gamma": 0.99}),
        "C": (torch.optim.lr_scheduler.StepLR, {"step_size": 2}),
    },
    scheduler_types={"per_step": ["G"], "per_epoch": ["C"]},
)
adapter = DANN(models=models, optimizers=optimizers, lr_schedulers=lr_schedulers)
print(adapter.lr_schedulers)

G: <torch.optim.lr_scheduler.ExponentialLR object at 0x7f5db619a1f0>
C: <torch.optim.lr_scheduler.StepLR object at 0x7f5db619a280>



If you don't wrap the adapter with a framework, then you have to step the lr schedulers manually as shown below.

(Here we're just demonstrating how the lr scheduler container works, so we're stepping it without computing a loss or stepping the optimizers etc.)

In [27]:
for epoch in range(4):
    for i in range(5):
        adapter.lr_schedulers.step("per_step")
    adapter.lr_schedulers.step("per_epoch")
    print(f"End of epoch={epoch}")
    print_optimizers_slim(adapter)

End of epoch=0
G: Adam with lr=0.9509900498999999
C: Adam with lr=1
D: Adam with lr=1
End of epoch=1
G: Adam with lr=0.9043820750088043
C: Adam with lr=0.1
D: Adam with lr=1
End of epoch=2
G: Adam with lr=0.8600583546412883
C: Adam with lr=0.1
D: Adam with lr=1
End of epoch=3
G: Adam with lr=0.8179069375972307
C: Adam with lr=0.010000000000000002
D: Adam with lr=1


### Training step

In [28]:
from pytorch_adapt.utils import common_functions as c_f

adapter.models.to(device)
data = c_f.batch_to_device(data, device)
loss = adapter.training_step(data)
print(loss)

{'total_loss': {'src_domain_loss': 3386.35498046875, 'target_domain_loss': 1900.6820068359375, 'c_loss': 0.0, 'total': 1762.345703125}}


### Inference

In [32]:
inference_data = torch.randn(32, 1000).to(device)
features, logits = adapter.inference(inference_data)
print(features.shape)
print(logits.shape)

torch.Size([32, 100])
torch.Size([32, 10])


### Passing in a custom inference function

In [34]:
def custom_inference_fn(cls):
    def return_fn(x):
        print("using custom_inference_fn")
        return cls.models["G"](x)

    return return_fn


adapter_custom = DANN(models=models, inference=custom_inference_fn)
features = adapter_custom.inference(inference_data)
print(features.shape)

using custom_inference_fn
torch.Size([32, 100])
